In [4]:
import numpy as np
from numpy import (isscalar, r_, log, around, unique, asarray,
                   zeros, arange, sort, amin, amax, any, atleast_1d,
                   sqrt, ceil, floor, array, compress,
                   pi, exp, ravel, count_nonzero, sin, cos, arctan2, hypot)
from collections import namedtuple
# z_stat dari fungsi?
# ====
# np.asarray?
# namedtuple?

In [ ]:
# PDF itu rumus untuk nyari probabilitasnya si variabel random secara umum
# misal diketahui nilai X-nya berapa, bisa langsung dimasukin ke PDF, nanti dapet nilai antara 0 dan 1, itu probabilitasnya

In [16]:
from scipy.stats import distributions
# to know p-val, understand 'two-sided','z_stat' first
distributions.norm.sf?
# pycharm
# sf refer ke method sf yang ada di rv_continuous 
# scipy.stats._distn_infrastructure.rv_continuous def sf(self,
#        x: Union[ndarray, Iterable, int, float],
#        *args: Any,
#        **kwds: Any) -> Any
def sf(self, x, *args, **kwds):    
    args, loc, scale = self._parse_args(*args, **kwds)
    _a, _b = self._get_support(*args)
    x, loc, scale = map(asarray, (x, loc, scale))
    args = tuple(map(asarray, args))
    dtyp = np.find_common_type([x.dtype, np.float64], [])
    x = np.asarray((x - loc)/scale, dtype=dtyp)
    cond0 = self._argcheck(*args) & (scale > 0)
    cond1 = self._open_support_mask(x, *args) & (scale > 0)
    cond2 = cond0 & (x <= _a)
    cond = cond0 & cond1
    output = zeros(shape(cond), dtyp)
    place(output, (1-cond0)+np.isnan(x), self.badvalue)
    place(output, cond2, 1.0)
    if np.any(cond):
        goodargs = argsreduce(cond, *((x,)+args))
        place(output, cond, self._sf(*goodargs))
    if output.ndim == 0:
        return output[()]
    return output

In [15]:
# sf = Survival function (1 - cdf) at x of the given RV.
# 1 - cdf
# cdf = cumulative distribution function
# distributions.norm diambil darinya norm
from scipy.stats import distributions
# pelajari z_statistik untuk wilcoxon?
def count_z_stat():
    T_val = 2
    se = 3
    mn = 2
    distr = 4
    z_stat = (T_val - mn - distr) / se
    return z_stat

def two_sided_p_val(z_stat):        
    prob = 2. * distributions.norm.sf(abs(z_stat))
    return prob
# main 
z_stat = count_z_stat()
print(two_sided_p_val(z_stat))

0.18242243945173575


In [12]:
# understand 'two-sided' first
def count_p_val(self, alternative, z_stat):
    if alternative == "two-sided":
        prob = 2. * distributions.norm.sf(abs(z_stat))
    elif alternative == "greater":
        # large T_val = r_plus indicates x is greater than y; i.e.
        # accept alternative in that case and return small p-value (sf)
        prob = distributions.norm.sf(z_stat)
    else:
        prob = distributions.norm.cdf(z_stat)
    return prob

In [16]:
correction=True
alternative="two-sided"
d = [10,21,23,41]
r_plus = 23
r_minus = 12
T = min(r_plus, r_minus)
mn = 0
if correction:
    if alternative == "two-sided":
        d = 0.5 * np.sign(T - mn)
    elif alternative == "less":
        d = -0.5
    else:
        d = 0.5
d        

0.5

In [13]:
def r_plus():
    count_r_plus = 1 
    return count_r_plus
def r_minus():
    count_r_plus = 2
    return count_r_plus 

def first_validation(zero_method,alternative):
    if zero_method not in ["wilcox", "pratt", "zsplit"]:
        raise ValueError("Zero method should be either 'wilcox' "
                         "or 'pratt' or 'zsplit'")

    if alternative not in ["two-sided", "less", "greater"]:
        raise ValueError("Alternative must be either 'two-sided', "
                         "'greater' or 'less'")
def check_y(y):        

In [14]:
WilcoxonResult = namedtuple('WilcoxonResult', ('statistic', 'pvalue'))

def wilcoxon(x, y=None, zero_method="wilcox", correction=False,
             alternative="two-sided"):
 
    first_validation(zero_method,alternative)
    if y is None:
        d = asarray(x)
        if d.ndim > 1:
            raise ValueError('Sample x must be one-dimensional.')
    else:
        x, y = map(asarray, (x, y))
        if x.ndim > 1 or y.ndim > 1:
            raise ValueError('Samples x and y must be one-dimensional.')
        if len(x) != len(y):
            raise ValueError('The samples x and y must have the same length.')
        d = x - y

    if zero_method in ["wilcox", "pratt"]:
        n_zero = np.sum(d == 0, axis=0)
        if n_zero == len(d):
            raise ValueError("zero_method 'wilcox' and 'pratt' do not work if "
                             "the x - y is zero for all elements.")

    if zero_method == "wilcox":
        # Keep all non-zero differences
        d = compress(np.not_equal(d, 0), d, axis=-1)

    count = len(d)
    if count < 10:
        warnings.warn("Sample size too small for normal approximation.")

    r = stats.rankdata(abs(d))
    r_plus = np.sum((d > 0) * r, axis=0)
    r_minus = np.sum((d < 0) * r, axis=0)

    if zero_method == "zsplit":
        r_zero = np.sum((d == 0) * r, axis=0)
        r_plus += r_zero / 2.
        r_minus += r_zero / 2.

    # return min for two-sided test, but r_plus for one-sided test
    # the literature is not consistent here
    # r_plus is more informative since r_plus + r_minus = count*(count+1)/2,
    # i.e. the sum of the ranks, so r_minus and the min can be inferred
    # (If alternative='pratt', r_plus + r_minus = count*(count+1)/2 - r_zero.)
    # [3] uses the r_plus for the one-sided test, keep min for two-sided test
    # to keep backwards compatibility
    if alternative == "two-sided":
        T = min(r_plus, r_minus)
    else:
        T = r_plus
    mn = count * (count + 1.) * 0.25
    se = count * (count + 1.) * (2. * count + 1.)

    if zero_method == "pratt":
        r = r[d != 0]
        # normal approximation needs to be adjusted, see Cureton (1967)
        mn -= n_zero * (n_zero + 1.) * 0.25
        se -= n_zero * (n_zero + 1.) * (2. * n_zero + 1.)

    replist, repnum = find_repeats(r)
    if repnum.size != 0:
        # Correction for repeated elements.
        se -= 0.5 * (repnum * (repnum * repnum - 1)).sum()

    se = sqrt(se / 24)

    # apply continuity correction if applicable
    d = 0
    if correction:
        if alternative == "two-sided":
            d = 0.5 * np.sign(T - mn)
        elif alternative == "less":
            d = -0.5
        else:
            d = 0.5

    # compute statistic and p-value using normal approximation
    z = (T - mn - d) / se
    if alternative == "two-sided":
        prob = 2. * distributions.norm.sf(abs(z))
    elif alternative == "greater":
        # large T = r_plus indicates x is greater than y; i.e.
        # accept alternative in that case and return small p-value (sf)
        prob = distributions.norm.sf(z)
    else:
        prob = distributions.norm.cdf(z)

    return WilcoxonResult(T, prob)
